In [1]:
# !pip install 'lightning-flash[image]'

import torch

import flash
# from flash.core.data.utils import download_data
from flash.image import ImageClassificationData, ImageClassifier

# 1. Create the DataModule
# download_data("https://pl-flash-data.s3.amazonaws.com/hymenoptera_data.zip", "./data")

datamodule = ImageClassificationData.from_folders(
    train_folder="./xarchive/vinted_dev_train",
    val_folder="./xarchive/vinted_dev_val",
    batch_size=4,
    transform_kwargs={"image_size": (100, 100), "mean": (0.485, 0.456, 0.406), "std": (0.229, 0.224, 0.225)},
)

# 2. Build the task
model = ImageClassifier(backbone="mobilenetv3_small_100", labels=datamodule.labels)

# 3. Create the trainer and finetune the model
trainer = flash.Trainer(max_epochs=3, gpus=torch.cuda.device_count())
#trainer = flash.Trainer(max_epochs=3, gpus=1)
#trainer = flash.Trainer(max_epochs=3, tpu_cores=8)
trainer.finetune(model, datamodule=datamodule, strategy="freeze")

# 4. Predict what's on a few images! ants or bees?
datamodule = ImageClassificationData.from_files(
    predict_files=[
        "./xarchive/vinted_dev_val/women__clothes__coats_and_jackets__coats__duster_coats/1373546971.jpeg",
        "./xarchive/vinted_dev_val/women__clothes__coats_and_jackets__coats__duster_coats/1375442944.jpeg",
        "./xarchive/vinted_dev_val/women__clothes__coats_and_jackets__coats__oversized_coats/1375442106.jpeg",
    ],
    batch_size=3,
)
predictions = trainer.predict(model, datamodule=datamodule, output="labels")
print(predictions)

# 5. Save the model!
trainer.save_checkpoint("image_classification_model.pt")

/home/becode2/Desktop/Zelus_Images/env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using 'mobilenetv3_small_100' provided by rwightman/pytorch-image-models (https://github.com/rwightman/pytorch-image-models).
No pretrained weights exist for this model. Using random initialization.


MisconfigurationException: GPUAccelerator can not run on your system since the accelerator is not available. The following accelerator(s) is available and can be passed into `accelerator` argument of `Trainer`: ['cpu'].